In [5]:
import math
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, fbeta_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from datetime import datetime
import xgboost as xgb
import json
warnings.filterwarnings('ignore')

In [6]:
train = pd.read_csv('./data/new_data/lgbtrain.csv')
test = pd.read_csv('./data/new_data/lgbtest.csv')
#seq_fea = ['launch_seq','playtime_seq','duration_prefer','interact_prefer']              
#样本多了一个，不好划分，删除最后一个样本。
train = train.iloc[:-1]

In [12]:
features = [i for i in train.columns if i not in ['label', 'user_id', 'end_date','launch_seq','playtime_seq','duration_prefer','interact_prefer']]  #将用户pid也加进去看看效果

y = train['label']
x = train[features]

new_test = test[features]
KF = StratifiedKFold(n_splits=5, random_state=2021, shuffle=True)
oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros((len(new_test)))

# 特征重要性
feat_imp_df = pd.DataFrame({'feat': features, 'imp': 0})

model = xgb.XGBRegressor( max_depth=10,
                           learning_rate=0.08,
                           n_estimators=10000,
                           subsample=0.8,
                           reg_alpha=0.5,
                           reg_lambda=0.5,
                           random_state=2021)
# 十折交叉验证
for fold_, (trn_idx, val_idx) in enumerate(KF.split(x,y)):
    print("##########第{}折############".format(fold_+1))

    Xtrain = x.iloc[trn_idx]
    Ytrain = y.iloc[trn_idx]

    X_val = x.iloc[val_idx]
    Y_val = y.iloc[val_idx]

    model = model.fit(Xtrain,
                      Ytrain,
                      eval_set=[(X_val, Y_val)],
                      verbose=100,
                      early_stopping_rounds=100
                    )

    feat_imp_df['imp'] += model.feature_importances_ / 5
    oof_xgb[val_idx] = model.predict(X_val)
    predictions_xgb += model.predict(new_test)

##########第1折############
[21:18:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "device", "feature_fraction", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1.94102
[100]	validation_0-rmse:1.27987
[192]	validation_0-rmse:1.28139
##########第2折############
[21:19:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "device", "feature_fraction", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please 

In [13]:
def aiyiqi_metric(y_true,y_pred):
    y_true = list(y_true)
    y_pred = list(y_pred)
    score = 0
    for i in range(len(y_true)):
        score += abs(y_true[i]-y_pred[i])/7
    return 100*(1-score/len(y_true))

In [15]:
score = aiyiqi_metric(y,oof_xgb)
print("aiqyiqi score: {}".format(score))

aiqyiqi score: 86.72500130316213


In [16]:
submit = test[['user_id']]
submit['pred'] = predictions_xgb/5
submit.columns = ['user_id', 'pred']
submit.to_csv("./data/submit/xgb_submit_{}.csv".format(score), index=False, header=False, float_format="%.2f")